In [1]:
import pandas as pd

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('max_colwidth', None)

artists_data_path = "../data/artists.jsonl"
sessions_data_path = "../data/sessions.jsonl"
track_storage_data_path = "../data/track_storage.jsonl"
tracks_data_path = "../data/tracks.jsonl"
users_data_path = "../data/users.jsonl"

artists_data_frame = pd.read_json(artists_data_path, lines=True)
sessions_data_frame = pd.read_json(sessions_data_path, lines=True)
tracks_data_frame = pd.read_json(tracks_data_path, lines=True)
users_data_frame = pd.read_json(users_data_path, lines=True)

# Mergowanie podobnych gatunków muzycznych (artists)
TODO


# Scalenie danych

In [2]:
merged_data_frame = pd.merge(sessions_data_frame, tracks_data_frame, left_on="track_id",
                             right_on="id")
merged_data_frame = pd.merge(merged_data_frame, users_data_frame, left_on="user_id",
                             right_on="user_id")
merged_data_frame = pd.merge(merged_data_frame, artists_data_frame, left_on="id_artist",
                             right_on="id")
merged_data_frame.head(5)


,timestamp,user_id,track_id,event_type,session_id,id_x,name_x,popularity,duration_ms,explicit,id_artist,release_date,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,name_y,city,street,favourite_genres,premium_user,id_y,name,genres
0,2023-01-03 05:09:55.000,101,2PmGtDUyJIpYBEtI1hQIVp,PLAY,124,2PmGtDUyJIpYBEtI1hQIVp,Ballrooms Of Mars,34,247707,0,3dBVyJ7JuOMt4GE9607Qin,1972-07-21,0.491,0.606,7,-10.172,0.0377,0.00327,0.000008,0.341,0.669,123.025,Klara Herdzik,Szczecin,pl. Floriana 55/22,"[permanent wave, mandopop, funk]",False,3dBVyJ7JuOMt4GE9607Qin,T. Rex,"[album rock, art rock, classic rock, folk rock, glam rock, protopunk, psychedelic rock, rock]"
1,2023-02-08 21:41:02.507,103,2PmGtDUyJIpYBEtI1hQIVp,PLAY,151,2PmGtDUyJIpYBEtI1hQIVp,Ballrooms Of Mars,34,247707,0,3dBVyJ7JuOMt4GE9607Qin,1972-07-21,0.491,0.606,7,-10.172,0.0377,0.00327,0.000008,0.341,0.669,123.025,Szymon Zawal,Gdynia,al. Głogowa 14/10,"[filmi, regional mexican, folk]",False,3dBVyJ7JuOMt4GE9607Qin,T. Rex,"[album rock, art rock, classic rock, folk rock, glam rock, protopunk, psychedelic rock, rock]"
2,2023-02-12 01:17:14.946,132,2PmGtDUyJIpYBEtI1hQIVp,PLAY,544,2PmGtDUyJIpYBEtI1hQIVp,Ballrooms Of Mars,34,247707,0,3dBVyJ7JuOMt4GE9607Qin,1972-07-21,0.491,0.606,7,-10.172,0.0377,0.00327,0.000008,0.341,0.669,123.025,Cezary Nóżka,Wrocław,aleja Olchowa 16,"[psychedelic rock, country rock, rock en espanol]",True,3dBVyJ7JuOMt4GE9607Qin,T. Rex,"[album rock, art rock, classic rock, folk rock, glam rock, protopunk, psychedelic rock, rock]"
3,2023-01-03 03:59:59.738,132,5yxYokipsWlpDCt4Th4VVc,PLAY,534,5yxYokipsWlpDCt4Th4VVc,Solid Gold Easy Action,35,140067,0,3dBVyJ7JuOMt4GE9607Qin,1973-03-16,0.449,0.749,9,-8.585,0.0775,0.01000,0.000000,0.391,0.448,106.861,Cezary Nóżka,Wrocław,aleja Olchowa 16,"[psychedelic rock, country rock, rock en espanol]",True,3dBVyJ7JuOMt4GE9607Qin,T. Rex,"[album rock, art rock, classic rock, folk rock, glam rock, protopunk, psychedelic rock, rock]"
4,2023-03-07 14:28:25.702,132,5yxYokipsWlpDCt4Th4VVc,PLAY,547,5yxYokipsWlpDCt4Th4VVc,Solid Gold Easy Action,35,140067,0,3dBVyJ7JuOMt4GE9607Qin,1973-03-16,0.449,0.749,9,-8.585,0.0775,0.01000,0.000000,0.391,0.448,106.861,Cezary Nóżka,Wrocław,aleja Olchowa 16,"[psychedelic rock, country rock, rock en espanol]",True,3dBVyJ7JuOMt4GE9607Qin,T. Rex,"[album rock, art rock, classic rock, folk rock, glam rock, protopunk, psychedelic rock, rock]"


## Wyliczenie, czy dana piosenka w danej sesji została pominięta

In [3]:
# delete rows with event_type other than PLAY and SKIP
merged_data_frame = merged_data_frame[merged_data_frame["event_type"].isin(["PLAY", "SKIP"])]

# TODO moze to trzeba liczyc inaczej?

# if for one user_id and track_id and session_id there is only PLAY event, then the track was not skipped
merged_data_frame["skipped"] = merged_data_frame.groupby(["user_id", "track_id", "session_id"])[
  "event_type"].transform(lambda x: x != "PLAY").astype(bool)

# if there are 2 rows with the same user_id, track_id and session_id, and different "skipped" value, remove the row with "skipped" = 0
merged_data_frame = merged_data_frame.drop_duplicates(subset=["user_id", "track_id", "session_id"],
                                                      keep="last")

merged_data_frame["number_of_matching_genres"] = merged_data_frame.apply(lambda x: len(set(x["genres"]).intersection(set(x["favourite_genres"]))), axis=1)

## Usunięcie niepotrzebnych kolumn

In [4]:
merged_data_frame = merged_data_frame.drop(
  columns=["id_x", "id_y", "premium_user", "name_x", "name_y", "city", "street", "id_artist",
           "event_type", "timestamp", "track_id", "session_id"])
merged_data_frame.head(10)

,user_id,popularity,duration_ms,explicit,release_date,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,favourite_genres,name,genres,skipped,number_of_matching_genres
0,101,34,247707,0,1972-07-21,0.491,0.606,7,-10.172,0.0377,0.00327,0.000008,0.341,0.669,123.025,"[permanent wave, mandopop, funk]",T. Rex,"[album rock, art rock, classic rock, folk rock, glam rock, protopunk, psychedelic rock, rock]",False,0
1,103,34,247707,0,1972-07-21,0.491,0.606,7,-10.172,0.0377,0.00327,0.000008,0.341,0.669,123.025,"[filmi, regional mexican, folk]",T. Rex,"[album rock, art rock, classic rock, folk rock, glam rock, protopunk, psychedelic rock, rock]",False,0
2,132,34,247707,0,1972-07-21,0.491,0.606,7,-10.172,0.0377,0.00327,0.000008,0.341,0.669,123.025,"[psychedelic rock, country rock, rock en espanol]",T. Rex,"[album rock, art rock, classic rock, folk rock, glam rock, protopunk, psychedelic rock, rock]",False,1
3,132,35,140067,0,1973-03-16,0.449,0.749,9,-8.585,0.0775,0.01000,0.000000,0.391,0.448,106.861,"[psychedelic rock, country rock, rock en espanol]",T. Rex,"[album rock, art rock, classic rock, folk rock, glam rock, protopunk, psychedelic rock, rock]",False,1
4,132,35,140067,0,1973-03-16,0.449,0.749,9,-8.585,0.0775,0.01000,0.000000,0.391,0.448,106.861,"[psychedelic rock, country rock, rock en espanol]",T. Rex,"[album rock, art rock, classic rock, folk rock, glam rock, protopunk, psychedelic rock, rock]",False,1
5,107,35,140067,0,1973-03-16,0.449,0.749,9,-8.585,0.0775,0.01000,0.000000,0.391,0.448,106.861,"[latin alternative, alternative rock, classic rock]",T. Rex,"[album rock, art rock, classic rock, folk rock, glam rock, protopunk, psychedelic rock, rock]",False,1
7,147,21,248360,0,1972-07-21,0.513,0.576,7,-9.704,0.0384,0.01130,0.000035,0.248,0.662,122.853,"[soft rock, hoerspiel, ranchera]",T. Rex,"[album rock, art rock, classic rock, folk rock, glam rock, protopunk, psychedelic rock, rock]",True,0
9,147,35,140067,0,1973-03-16,0.449,0.749,9,-8.585,0.0775,0.01000,0.000000,0.391,0.448,106.861,"[soft rock, hoerspiel, ranchera]",T. Rex,"[album rock, art rock, classic rock, folk rock, glam rock, protopunk, psychedelic rock, rock]",True,0
12,147,35,140067,0,1973-03-16,0.449,0.749,9,-8.585,0.0775,0.01000,0.000000,0.391,0.448,106.861,"[soft rock, hoerspiel, ranchera]",T. Rex,"[album rock, art rock, classic rock, folk rock, glam rock, protopunk, psychedelic rock, rock]",True,0
15,147,35,140067,0,1973-03-16,0.449,0.749,9,-8.585,0.0775,0.01000,0.000000,0.391,0.448,106.861,"[soft rock, hoerspiel, ranchera]",T. Rex,"[album rock, art rock, classic rock, folk rock, glam rock, protopunk, psychedelic rock, rock]",True,0


In [5]:
merged_data_frame.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9881 entries, 0 to 16822
Data columns (total 20 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   user_id                    9881 non-null   int64  
 1   popularity                 9881 non-null   int64  
 2   duration_ms                9881 non-null   int64  
 3   explicit                   9881 non-null   int64  
 4   release_date               9881 non-null   object 
 5   danceability               9881 non-null   float64
 6   energy                     9881 non-null   float64
 7   key                        9881 non-null   int64  
 8   loudness                   9881 non-null   float64
 9   speechiness                9881 non-null   float64
 10  acousticness               9881 non-null   float64
 11  instrumentalness           9881 non-null   float64
 12  liveness                   9881 non-null   float64
 13  valence                    9881 non-null   float64
 

In [6]:
merged_data_frame.describe()

,user_id,popularity,duration_ms,explicit,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,number_of_matching_genres
count,9881.000000,9881.000000,9.881000e+03,9881.000000,9881.000000,9881.000000,9881.00000,9881.000000,9881.000000,9881.000000,9881.000000,9881.000000,9881.000000,9881.000000,9881.000000
mean,123.952434,44.646594,2.310971e+05,0.080963,0.563325,0.600228,5.25463,-9.125568,0.083663,0.328242,0.067860,0.207252,0.539190,120.525376,0.147860
std,14.826553,25.085211,9.345541e+04,0.272793,0.165085,0.230087,3.55038,4.286563,0.121603,0.311197,0.199239,0.192774,0.250657,28.640296,0.375486
min,101.000000,0.000000,3.100000e+04,0.000000,0.064000,0.000592,0.00000,-35.898000,0.022700,0.000001,0.000000,0.016000,0.002280,36.241000,0.000000
25%,110.000000,25.000000,1.803480e+05,0.000000,0.453000,0.428000,2.00000,-11.515000,0.033500,0.042500,0.000000,0.095100,0.338000,98.231000,0.000000
50%,125.000000,39.000000,2.158000e+05,0.000000,0.570000,0.624000,5.00000,-8.446000,0.042800,0.223000,0.000044,0.127000,0.544000,120.031000,0.000000
75%,136.000000,75.000000,2.626530e+05,0.000000,0.683000,0.794000,9.00000,-6.114000,0.077100,0.579000,0.005030,0.251000,0.742000,138.023000,0.000000
max,150.000000,96.000000,2.365307e+06,1.000000,0.975000,0.998000,11.00000,1.716000,0.965000,0.996000,0.978000,0.997000,0.991000,209.231000,3.000000


In [7]:
from keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn import metrics
from scipy.stats import entropy

df = merged_data_frame.copy()

df["encoded_genres"] = df["genres"].apply(lambda x: " ".join(x)) + df["favourite_genres"].apply(lambda x: " ".join(x))


print("Entropy of skipped column: ", str(round(entropy(df["skipped"].value_counts()), 4)))

print("MI(skipped, number_of_matching_genres): ", str(round(metrics.mutual_info_score(df["skipped"], df["number_of_matching_genres"]), 4)))

df.head(5)

Entropy of skipped column:  0.657
MI(skipped, number_of_matching_genres):  0.0001


,user_id,popularity,duration_ms,explicit,release_date,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,favourite_genres,name,genres,skipped,number_of_matching_genres,encoded_genres
0,101,34,247707,0,1972-07-21,0.491,0.606,7,-10.172,0.0377,0.00327,0.000008,0.341,0.669,123.025,"[permanent wave, mandopop, funk]",T. Rex,"[album rock, art rock, classic rock, folk rock, glam rock, protopunk, psychedelic rock, rock]",False,0,album rock art rock classic rock folk rock glam rock protopunk psychedelic rock rockpermanent wave mandopop funk
1,103,34,247707,0,1972-07-21,0.491,0.606,7,-10.172,0.0377,0.00327,0.000008,0.341,0.669,123.025,"[filmi, regional mexican, folk]",T. Rex,"[album rock, art rock, classic rock, folk rock, glam rock, protopunk, psychedelic rock, rock]",False,0,album rock art rock classic rock folk rock glam rock protopunk psychedelic rock rockfilmi regional mexican folk
2,132,34,247707,0,1972-07-21,0.491,0.606,7,-10.172,0.0377,0.00327,0.000008,0.341,0.669,123.025,"[psychedelic rock, country rock, rock en espanol]",T. Rex,"[album rock, art rock, classic rock, folk rock, glam rock, protopunk, psychedelic rock, rock]",False,1,album rock art rock classic rock folk rock glam rock protopunk psychedelic rock rockpsychedelic rock country rock rock en espanol
3,132,35,140067,0,1973-03-16,0.449,0.749,9,-8.585,0.0775,0.01000,0.000000,0.391,0.448,106.861,"[psychedelic rock, country rock, rock en espanol]",T. Rex,"[album rock, art rock, classic rock, folk rock, glam rock, protopunk, psychedelic rock, rock]",False,1,album rock art rock classic rock folk rock glam rock protopunk psychedelic rock rockpsychedelic rock country rock rock en espanol
4,132,35,140067,0,1973-03-16,0.449,0.749,9,-8.585,0.0775,0.01000,0.000000,0.391,0.448,106.861,"[psychedelic rock, country rock, rock en espanol]",T. Rex,"[album rock, art rock, classic rock, folk rock, glam rock, protopunk, psychedelic rock, rock]",False,1,album rock art rock classic rock folk rock glam rock protopunk psychedelic rock rockpsychedelic rock country rock rock en espanol


In [8]:
# save to jsonl file
merged_data_frame.to_json("../data/merged_data.jsonl", orient="records", lines=True)
